In [ ]:
!pip install colabcode
!pip install fastapi

In [ ]:
from colabcode import ColabCode
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware


In [ ]:
cc= ColabCode(port=12000,code=False)

In [ ]:
app=FastAPI()

@app.get("/")
async def read_root():
  return {"Hello":"World"}

In [ ]:
cc.run_app(app=app)

INFO:     Started server process [63]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://7a72-35-194-238-172.ngrok.io" -> "http://localhost:12000"
INFO:     112.134.65.111:0 - "GET / HTTP/1.1" 200 OK
INFO:     112.134.65.111:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [63]


In [ ]:
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import OneClassSVM
from numpy import where

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import OneClassSVM
from numpy import where

app=FastAPI(title='The cheating Predictor')

origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=['OPTION','GET', 'POST'],
    allow_headers=["*"],
)

class Data(BaseModel):
    x_min:int
    x_max:int
    y_max:int
    y_min:int
    gazes: list


@app.post("/api",tags=["prediction"])
async def predict(data:Data):
  df =  pd.DataFrame(data.gazes,columns=["X", "Y"])
  df['X']=df.X.astype(float)
  df['Y']=df.Y.astype(float)
  print(df.shape)
  x_min,x_max,y_max,y_min= data.x_min, data.x_max, data.y_max, data.y_min
  data = {'status': [], 'elapased_time': []}
  df_new = pd.DataFrame(data)

  status=-1
  c=0
  k=0

  for index, row in df.iterrows():
    if (row['X']>x_max or row['X']<x_min or row['Y']>y_max or row['Y']<y_min):
      if status==0:
        df_new=df_new.append({'status':0,'elapased_time':c},ignore_index=True)
        c=0
      status= 1
      c=c+1
    else:
      if status==1:
        df_new=df_new.append({'status':1,'elapased_time':c},ignore_index=True)
        c=0
      status= 0
      c=c+1
  df_new=df_new.append({'status':status,'elapased_time':c},ignore_index=True)
  new_dtypes = {"status": int, "elapased_time": int}
  df_new = df_new.astype(new_dtypes)

  svm = OneClassSVM(kernel='rbf', gamma=0.001, nu=0.1)

  svm.fit(df_new)
  pred = svm.predict(df_new)

  anom_index = where(pred==-1)
  df_new = df_new.reset_index(drop=True) 
  values = df_new.loc[anom_index]
  df_new['svm_p']=pred
  # print(values.shape[0])
  result='non-cheat'

  for index, row in df_new.iterrows():
    if(row['svm_p']==-1 and row['status']==0):
      if k>15:
        print(k)
        result='cheat'
        break
      k=0
    else:
      k=k+1
  
  # print(x_min,x_max,y_max,y_min)
  return {"result":result}

In [ ]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://898d-104-196-251-164.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [64]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     112.134.65.157:0 - "OPTIONS /api HTTP/1.1" 200 OK
(3000, 2)
INFO:     112.134.65.157:0 - "POST /api HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [64]


Some Tests

In [ ]:
-57,1677,923,24

In [ ]:
df =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Research/Test Data/webquiz_cheat1.csv',names=["X", "Y"])
predict(-6, 1505, 839, 27,df)

29
cheat


In [ ]:
df =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Research/Test Data/webquiz_cheat.csv',names=["X", "Y"])
predict(-26 ,1332,822,23,df)

25
cheat


In [ ]:
df =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Research/Test Data/webquiz_normal.csv',names=["X", "Y"])
predict(49, 1399, 817 ,65,df)

non-cheat


In [ ]:
df =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Research/Test Data/webquiz_normal1.csv',names=["X", "Y"])
predict(49, 1399, 817 ,65 ,df)

non-cheat


In [ ]:
df =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Research/Test Data/webquiz_good.csv',names=["X", "Y"])
predict( -26.0,1332,822, 23,df)

non-cheat
